In [ ]:
import pandas as pd
import numpy as np

import psycopg2
from psycopg2 import errorcodes, OperationalError

In [28]:
gastos = pd.read_csv('..\datos\gastos.csv')

In [29]:
gastos.head()

,año,ncodi,totalcompra,producfarma,materialsani,implantes,restomateriasani,servcontratado,trabajocontratado,xrestocompras,...,xrestoserviexter,gastopersonal,sueldos,indemnizacion,segsocempresa,otrgassocial,dotaamortizacion,perdidadeterioro,xrestogasto,totcompragasto
0,2020,113.0,1535894,344330.0,169443,0.0,169443.0,1877.0,678201.0,342043.0,...,506018.0,26209836,19875799,349499.0,5984538,0.0,298707.0,-6504.0,50165.0,28810972
1,2020,619.0,7884157,971762.0,1749006,493444.0,1255562.0,3800207.0,1248674.0,114508.0,...,2635675.0,7083969,5203283,151380.0,1591074,138232.0,2423752.0,0.0,312984.0,20573925
2,2020,163.0,5453290,126493.0,1412366,574127.0,838239.0,3217684.0,647852.0,48895.0,...,882274.0,2890388,2180177,-402.0,663398,47215.0,202688.0,-54028.0,232471.0,9677715
3,2020,131.0,245802,72318.0,52506,0.0,52506.0,1114.0,24303.0,95561.0,...,487053.0,1887375,1437421,9506.0,416398,24050.0,12084.0,240.0,132044.0,2844476
4,2020,375.0,762218,114702.0,106844,60.0,106784.0,14526.0,188463.0,337683.0,...,192132.0,7764302,5838470,0.0,1837175,88657.0,184413.0,93708.0,1552.0,9037657


In [30]:
ingresos = pd.read_csv('..\datos\ingresos.csv', index_col=0)

In [31]:
ingresos.head()

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,tipo,año,ncodi
0,0.0,103677.0,64383.0,39294.0,61456.0,Hospital,2013,"990,00"
1,324687.0,478280.0,392084.0,86196.0,484228.0,Hospital,2013,"680,00"
2,793512.0,1803352.0,1803352.0,0.0,20402.0,Hospital,2013,"411,00"
3,0.0,0.0,0.0,0.0,0.0,Hospital,2013,"241,00"
4,24406.0,63557.0,55091.0,8466.0,44106.0,Hospital,2013,"558,00"


In [32]:
columnas_gastos = gastos.columns.values
columnas_gastos[1] = 'id_hospital'
columnas_gastos[np.where(columnas_gastos == 'año')] = 'anio'
columnas_gastos

array(['anio', 'id_hospital', 'totalcompra', 'producfarma',
       'materialsani', 'implantes', 'restomateriasani', 'servcontratado',
       'trabajocontratado', 'xrestocompras', 'variaexistencias',
       'servexteriores', 'sumistro', 'xrestoserviexter', 'gastopersonal',
       'sueldos', 'indemnizacion', 'segsocempresa', 'otrgassocial',
       'dotaamortizacion', 'perdidadeterioro', 'xrestogasto',
       'totcompragasto'], dtype=object)

In [33]:
columnas_ingresos = ingresos.columns.values
columnas_ingresos[-1] = 'id_hospital'
columnas_ingresos[np.where(columnas_ingresos == 'tipo')] = 'id_tipo_hosp'
columnas_ingresos[np.where(columnas_ingresos == 'año')] = 'anio'
columnas_ingresos

array(['particulares', 'aseguradoras', 'aseguradoras_enfermedad',
       'aseguradoreas_trafico', 'mutuas', 'id_tipo_hosp', 'anio',
       'id_hospital'], dtype=object)

In [34]:
columnas_tipo = np.array(['id_tipo_hosp', 'tipo_hosp'])
columnas_tipo

array(['id_tipo_hosp', 'tipo_hosp'], dtype='<U12')

In [35]:
columnas_hosp = np.array("id_hospital")
columnas_hosp

array('id_hospital', dtype='<U11')

In [36]:
def conectar(ddbb):
    try:
        connection = psycopg2.connect(
            database = ddbb,
            user = "postgres",
            password = "admin",
            host = "localhost",
            port = "5432"
        )
    except OperationalError as e:
        if e.pgcode == errorcodes.INVALID_PASSWORD:
            print("La constraseña es errónea.")
        elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
            print("Error de conexión")
        else:
            print(f'Error:{e}')
    return connection

def query_fetch(connection, query_text):
    cursor = connection.cursor()
    cursor.execute(query_text)
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    return result

def query_commit(connection, query_text, *valores):
    cursor = connection.cursor()
    cursor.execute(query_text, *valores)
    connection.commit()
    cursor.close()
    connection.close()
    return print("Done!")

def query_commit_many(connection, query_text, *valores):
    cursor = connection.cursor()
    cursor.executemany(query_text, *valores)
    connection.commit()
    cursor.close()
    connection.close()
    return print("Done!")

In [38]:
# with open(r"C:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Retos\team-proyecto-bbdd\sql\script_creacion_bdd.sql") as r:
#     creator = r.read()

# conexion = conectar('HOSPITALES')
# query_commit(conexion, creator)

In [39]:
# conexion = conectar(None)
# query_commit(conexion, query_text="CREATE DATABASE IF NOT EXISTS FOOO;")

In [41]:
import sys
sys.path.append("..")
from src.soporte_queries_creacion import *


In [ ]:
'''insert into hospitales(ncodi)
                                values (%s)'''
